In [1]:
###########################
### MODULES NECESSAIRES ###
###########################


# Import the necessary libraries
import numpy as np
import pandas as pd
import math
import fonctionsSupervisedLearning2 as fsl2

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn import svm

from auxFonctions import AminoAcid

/var/folders/wp/4cr2jl295bs4z4czxy_p6w5m0000gn/T/ipykernel_76699/86324599.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/Users/mathiasperez/Documents/GitHub/442-2-Protein-cleavage/fonctionsSupervisedLearning2.py:120: SyntaxWarning: invalid escape sequence '\ '
  '''


In [2]:
# Read data from a file into a list of entries
with open('data/SIG_13.red', 'r') as file:
    entries = file.read().split('\n   ')

In [3]:
##############################
## RECUPERATION DES DONNÉES ##
##############################

# Process each entry
processed_entries = [fsl2.process_entry(entry) for entry in entries]

# Create a DataFrame
df = pd.DataFrame(processed_entries)

# Get the position of the cleavage site
cleavage_site_position = df['Annotation'].apply(lambda x: x.find('C'))

# Split the primary structure into a list of amino acids
amino_acid_seq = df['Primary Structure'].apply(lambda x: list(x))

## Traitement des données

Put words in vector and vice versa

In [4]:
df_exploitable = fsl2.convert_df_to_vectors2(df)

In [5]:
n = 15
p,q = 13, 2  # p+q = 15

In [6]:
# Create a DataFrame to store, for each primary structure, the neihborhood of the cleavage site
# The neighborhood is defined as the word of length p+q starting p letters before the cleavage site
correct_neighborhood = pd.Series()
for i, seq in amino_acid_seq.items():
    correct_neighborhood[i] = ''.join(seq[cleavage_site_position[i]-p:cleavage_site_position[i]+q])

In [18]:
print(correct_neighborhood.get(0))

LLAFTLLFATCIARH


## Train the models

In [8]:
#svm_model_nei, accuracy_nei = fsl2.create_model2(15, df_exploitable, random_state=42, nb_letters = 26, kernel_neighbor= 'rbf',C_nei = 1)

df_exploitable_100 = df_exploitable.iloc[:,1:100]

C_list = [0.1,1,2]
kernel_list = ['linear', 'rbf', 'sigmoid', 'poly']
n_list = [15, 17, 19]
results = []

for C_nei in C_list:
    for kernel_nei in kernel_list:
        for n in n_list:
            svm_model_nei, accuracy_nei = fsl2.create_model2(n, df_exploitable.iloc[:,1:100], kernel_nei, C_nei)
            results.append((C_nei, kernel_nei, n, accuracy_nei))

results_df = pd.DataFrame(results, columns=['C_nei','kernel_nei', 'n', 'accuracy_nei'])
results_df.to_csv('results2.csv', index=True)


In [10]:
best_results = results_df.loc[(results_df['accuracy_nei']).idxmax()]
print(best_results)

C_nei                1.0
kernel_nei        linear
n                     19
accuracy_nei    0.989362
Name: 14, dtype: object


In [11]:
svm_model_nei, accuracy_nei = fsl2.create_model2(15, df_exploitable,'linear', 0.1)

In [ ]:
count = 0
for i in range(0, len(correct_neighborhood)):
    X = fsl2.word_to_vector(correct_neighborhood.get(4))
    print(fsl2.find_cleavage2(X,svm_model_nei,15))


In [51]:
# print(fsl.find_cleavage(test, threshold = 0.1))
pos_pred = np.array([fsl.find_cleavage(X,svm_model_in, svm_model_pos, threshold = 0.0, n= 17) for X in df_exploitable['P_Structure_vector'][:100]])

in_pred = 1-np.isnan(pos_pred)

pos_pred[in_pred == 0] = 0

print(accuracy_score(df_exploitable["Annotation_pos"][:100], pos_pred))

0.53
